In [178]:
import pandas as pd
import matplotlib

In [179]:
data = pd.read_csv('data/train_data/train_task_3_4.csv')
print(len(data))
data.head()

1382727


,QuestionId,UserId,AnswerId,IsCorrect,CorrectAnswer,AnswerValue
0,898,2111,280203,1,2,2
1,767,3062,55638,1,3,3
2,165,1156,386475,1,2,2
3,490,1653,997498,1,4,4
4,298,3912,578636,1,3,3


In [180]:
answer_metadata = pd.read_csv('data/metadata/answer_metadata_task_3_4.csv')
data=data.merge(answer_metadata, on="AnswerId", how="left")
data=data.groupby("QuestionId")[["Confidence","IsCorrect"]].mean()

# # Create avg confidence
# data=data.dropna(subset=["Confidence","IsCorrect"]) => Cause poor result
data['Confidence'].fillna(value=data['Confidence'].mean(),inplace=True)
data['IsCorrect'].fillna(value=data['IsCorrect'].mean(),inplace=True)

data

,Confidence,IsCorrect
QuestionId,,
0,66.176471,0.443457
1,74.036219,0.571429
2,66.201354,0.385214
3,85.021246,0.808757
4,59.343434,0.401408
...,...,...
943,83.793687,0.566528
944,69.852941,0.142857
945,72.244733,0.422336


In [181]:

data=data.sort_values("Confidence",ascending=True).reset_index()
# data=data.sort_values("Confidence",ascending=False).reset_index()
# data=data.sort_values("IsCorrect",ascending=True).reset_index()
# data=data.sort_values("IsCorrect",ascending=False).reset_index()

# data['temp'] = 1-data['IsCorrect']
# data["Quality"] = data[['Confidence','temp']].min(axis=1)/data[['Confidence','temp']].max(axis=1)
# data=data.sort_values("Quality",ascending=False).reset_index()

data


,QuestionId,Confidence,IsCorrect,temp,Quality
0,548,25.000000,0.400000,0.600000,0.024000
1,169,47.910373,0.261946,0.738054,0.015405
2,775,45.000000,0.325000,0.675000,0.015000
3,442,50.000000,0.250000,0.750000,0.015000
4,273,47.286822,0.292683,0.707317,0.014958
...,...,...,...,...,...
943,422,92.968750,0.895833,0.104167,0.001120
944,924,93.435155,0.923497,0.076503,0.000819
945,847,74.036219,1.000000,0.000000,0.000000
946,841,74.036219,1.000000,0.000000,0.000000


In [182]:
eval_validation = pd.read_csv('data/test_data/quality_response_remapped_public.csv')
print(len(eval_validation))
eval_validation.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,0,0,909,318,2,2,2,1.0,2
1,16,16,285,609,2,2,2,2.0,2
2,48,48,132,271,1,2,2,2.0,1
3,38,38,704,127,2,2,1,2.0,2
4,33,33,16,586,1,1,2,2.0,2


In [183]:
eval_test = pd.read_csv('data/test_data/quality_response_remapped_private.csv')
print(len(eval_test))
eval_test.head()

25


,Unnamed: 0.1,Unnamed: 0,left,right,T1_ALR,T2_CL,T3_GF,T4_MQ,T5_NS
0,15,15,145,762,2,2,2,2.0,2
1,31,31,307,208,1,1,1,2.0,1
2,25,25,278,195,1,1,1,1.0,1
3,3,3,13,153,2,2,2,2.0,2
4,40,40,23,100,2,1,1,1.0,1


In [184]:
submission = pd.read_csv('submission/template.csv')
print(len(submission))
print(submission.head())
for idx in range(len(data)):
    tmp = data[data['QuestionId']==idx].index
    submission.loc[idx,'ranking'] = 987654321+idx if tmp.empty else tmp[0]
print(data[710:720])
submission.head()

948
   QuestionId  ranking
0           0      NaN
1           1      NaN
2           2      NaN
3           3      NaN
4           4      NaN
     QuestionId  Confidence  IsCorrect      temp   Quality
710         348   80.339806   0.611155  0.388845  0.004840
711         585   76.075731   0.632364  0.367636  0.004832
712         285   72.151163   0.651419  0.348581  0.004831
713         714   82.077922   0.604400  0.395600  0.004820
714         170   76.837270   0.630759  0.369241  0.004805
715         337   80.513017   0.613499  0.386501  0.004800
716         889   74.322799   0.644397  0.355603  0.004785
717         222   83.344875   0.601316  0.398684  0.004784
718         721   82.729008   0.604527  0.395473  0.004780
719         939   82.102273   0.608205  0.391795  0.004772


,QuestionId,ranking
0,0,274.0
1,1,598.0
2,2,200.0
3,3,919.0
4,4,136.0


In [185]:
def order2preference(ord, QL, QR):
    preference = []
    for idx in range(len(QL)):
        rankL = ord[ord.QuestionId==QL[idx]].ranking.values[0]
        rankR = ord[ord.QuestionId==QR[idx]].ranking.values[0]
        preference.append(1 if rankL<rankR else 2)
    return preference

def evaluate(truth, submission):
    QL = list(truth.left)
    QR = list(truth.right)
    if len(QL) != len(QR):
        message = 'left and right lengths are not the same'
        exit(message)
    print(list(truth["preference"]))
    return sum(x==y for x,y in zip(truth["preference"],order2preference(submission,QL,QR)))/len(QL)

def evaluates(truth, submission):
    return [
        evaluate(truth.rename(columns={"T1_ALR":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T2_CL":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T3_GF":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T4_MQ":"preference"})[["left","right","preference"]], submission),
        evaluate(truth.rename(columns={"T5_NS":"preference"})[["left","right","preference"]], submission)]


In [186]:
list_vali = evaluates(eval_validation,submission)
print(list(order2preference(submission,list(eval_validation.left),list(eval_validation.right))))
print(max(list_vali))
print(list_vali)

[2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 2, 1]
[2, 2, 2, 2, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 2, 2, 1, 1, 1, 2, 1, 1, 1]
[2, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 2, 1, 1]
[1.0, 2.0, 2.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0, 2.0, 2.0, 2.0, 1.0, 2.0, 2.0, 2.0, 1.0, 1.0]
[2, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 1, 2, 1, 2, 1]
[2, 2, 2, 2, 2, 2, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 1, 2, 1, 2, 1]
0.8
[0.76, 0.72, 0.8, 0.6, 0.76]


In [187]:
list_test = evaluates(eval_test,submission)
print(max(list_test))
print(list_test)
print(list(order2preference(submission,list(eval_test.left),list(eval_test.right))))

[2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 1, 2, 1, 2]
[2, 1, 1, 2, 1, 2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 1, 2]
[2.0, 2.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 1.0, 2.0, 2.0, nan, 1.0, 1.0, 2.0, 1.0, 2.0, 1.0, 1.0, 2.0, 2.0, 1.0, 1.0, 1.0, 2.0]
[2, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 2, 2, 2, 1, 2]
0.68
[0.68, 0.56, 0.56, 0.56, 0.68]
[2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 2]
